# Task 2: Text Chunking, Embedding, and Vector Store Indexing

This notebook implements the complete RAG data preparation pipeline:
- Stratified sampling from the cleaned complaint dataset
- Configurable text chunking
- Embedding generation using sentence-transformers
- ChromaDB vector store creation with metadata
- Persistence of the vector store under vector_store/


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [4]:
import os
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

import chromadb
from chromadb.config import Settings


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [6]:
# Chunking configuration
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)


In [10]:
import os
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\Yosef Maregn\Desktop\rag-complaint-chatbot\notebooks


In [11]:
import pandas as pd
import os

DATA_PATH = "../data/processed/filtered_complaints.csv"

# Verify the file exists first
assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}"

df = pd.read_csv(DATA_PATH)

print("Dataset shape:", df.shape)
print("Products available:\n", df["Product"].value_counts())


Dataset shape: (82164, 20)
Products available:
 Product
Credit card        80667
Money transfers     1497
Name: count, dtype: int64


In [12]:
from sklearn.model_selection import train_test_split

# Define sample size (adjust as needed)
SAMPLE_SIZE = 12000

# Stratified sampling based on 'Product'
df_sample, _ = train_test_split(
    df,
    train_size=SAMPLE_SIZE,
    stratify=df["Product"],
    random_state=42
)

print("Sample shape:", df_sample.shape)
print("Product distribution in sample:\n", df_sample["Product"].value_counts())


Sample shape: (12000, 20)
Product distribution in sample:
 Product
Credit card        11781
Money transfers      219
Name: count, dtype: int64


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm

# Chunking configuration
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

# Prepare lists for chunks and metadata
texts = []
metadatas = []

# Iterate over sampled complaints
for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    narrative = row["cleaned_narrative"]
    
    if pd.isna(narrative) or narrative.strip() == "":
        continue  # skip empty narratives

    chunks = text_splitter.split_text(narrative)

    for idx, chunk in enumerate(chunks):
        texts.append(chunk)
        metadatas.append({
            "complaint_id": row["Complaint ID"],
            "product": row["Product"],
            "issue": row["Issue"],
            "chunk_index": idx,
            "total_chunks": len(chunks)
        })

print("Total chunks created:", len(texts))


100%|██████████| 12000/12000 [00:09<00:00, 1330.70it/s]

Total chunks created: 33626


In [14]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load embedding model
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Batch size (safe to avoid memory issues)
BATCH_SIZE = 500

embeddings = []

for i in tqdm(range(0, len(texts), BATCH_SIZE)):
    batch_texts = texts[i:i+BATCH_SIZE]
    batch_embeddings = embedding_model.encode(batch_texts, show_progress_bar=False)
    embeddings.append(batch_embeddings)

# Combine all batches into a single array
embeddings = np.vstack(embeddings)

print("Embeddings shape:", embeddings.shape)


100%|██████████| 68/68 [1:02:29<00:00, 55.14s/it]

Embeddings shape: (33626, 384)
